<a href="https://colab.research.google.com/github/MA18A/Quiz2-3/blob/main/1820507_GUI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from PyQt5.QtWidgets import QApplication, QWidget, QLabel, QSlider, QHBoxLayout, QVBoxLayout, QPushButton, QFileDialog, QGridLayout
from PyQt5.QtGui import QPixmap, QImage
from PyQt5.QtCore import Qt
import numpy as np
import cv2
import sys

ModuleNotFoundError: ignored

In [ ]:
class DetectCircles(QWidget):
    def __init__(self):
        super().__init__()
        self.initUI()
        
    def initUI(self):
        # To create labels to display the images:
        self.image_label = QLabel(self)
        self.edge_label = QLabel(self)
        
        # To create a label for the title:
        self.title_label = QLabel('Finding circles in images using Hough Transform')
        self.title_label.setAlignment(Qt.AlignCenter)  # Center the title label
        
        # To create sliders to control the parameters of Hough transform:
        self.param1_slider = QSlider(Qt.Horizontal)
        self.param1_slider.setMinimum(1)
        self.param1_slider.setMaximum(255)
        self.param1_slider.setValue(50)
        self.param2_slider = QSlider(Qt.Horizontal)
        self.param2_slider.setMinimum(1)
        self.param2_slider.setMaximum(50)
        self.param2_slider.setValue(30)
        
        # To create a button to load the image:
        self.load_button = QPushButton('Load Image')
        self.load_button.clicked.connect(self.load_image)
        
        # To create a button to detect edges:
        self.detect_button = QPushButton('Detect Edges')
        self.detect_button.clicked.connect(self.detect)
        
        # To create a layout for the sliders and buttons:
        hbox = QHBoxLayout()
        hbox.addWidget(QLabel('Param1'))
        hbox.addWidget(self.param1_slider)
        hbox.addWidget(QLabel('Param2'))
        hbox.addWidget(self.param2_slider)
        
        # CreTo createte a grid layout for the images:
        grid = QGridLayout()
        grid.addWidget(self.image_label, 0, 0)
        grid.addWidget(self.edge_label, 0, 1)
        
        vbox = QVBoxLayout()
        vbox.addWidget(self.title_label)  # To add the title label:
        vbox.addLayout(hbox)
        vbox.addWidget(self.load_button)
        vbox.addWidget(self.detect_button)
        vbox.addLayout(grid)
        
        # To set the layout of the widget:
        self.setLayout(vbox)
        # To set the window properties
        self.setGeometry(300, 300, 600, 600)
        self.setWindowTitle('Circular Hough Transform')
        self.show()
    
    def load_image(self):
        # To open a file dialog to select the image:
        options = QFileDialog.Options()
        options |= QFileDialog.ReadOnly
        file_name, _ = QFileDialog.getOpenFileName(self,"Select an image", "","Images (*.png *.xpm *.jpg *.bmp);;All Files (*)", options=options)
        if file_name:
            # To load the image:
            self.image = cv2.imread(file_name)
            # To convert the image to RGB:
            self.image = cv2.cvtColor(self.image, cv2.COLOR_BGR2RGB)
    
    def detect(self):
        # To clear the image_label widget:
        self.image_label.clear()
        # To preform Canny edge detection:
        self.edges = cv2.Canny(self.image, self.param1_slider.value(), self.param1_slider.value() * 3)
        
        self.image_label.clear()
        
        # To preform Hough transform to detect circles:
        circles = cv2.HoughCircles(self.edges, cv2.HOUGH_GRADIENT, 1, 20, param1=self.param1_slider.value(), param2=self.param2_slider.value(), minRadius=5, maxRadius=30)
        
        # To draw the circles on the image:
        if circles is not None:
            circles = np.uint16(np.around(circles))
            for i in circles[0,:]:
                # To draw the outer circle:
                cv2.circle(self.image, (i[0], i[1]), i[2], (0, 255, 0), 2)
                # To draw the center of the circle:
                cv2.circle(self.image, (i[0], i[1]), 2, (0, 0, 255), 3)
        
        # To convert the edges image to grayscale:
        edges_gray = cv2.cvtColor(self.edges, cv2.COLOR_GRAY2RGB)
        # Set the size of the images:
        self.image_label.setFixedSize(self.image.shape[1], self.image.shape[0])
        self.edge_label.setFixedSize(self.edges.shape[1], self.edges.shape[0])
        
        # To display the images:
        height, width, _ = self.image.shape
        qt_image = QImage(self.image.data, width, height, width * 3, QImage.Format_RGB888).rgbSwapped()
        pixmap = QPixmap.fromImage(qt_image)
        self.image_label.setPixmap(pixmap)
        self.image_label.setAlignment(Qt.AlignCenter)
        height, width, _ = edges_gray.shape
        qt_image = QImage(edges_gray.data, width, height, width * 3, QImage.Format_RGB888).rgbSwapped()
        pixmap = QPixmap.fromImage(qt_image)
        self.edge_label.setPixmap(pixmap)
        self.edge_label.setAlignment(Qt.AlignCenter)

In [ ]:
if __name__ == '__main__':
    app = QApplication(sys.argv)
    ex = DetectCircles()
    sys.exit(app.exec_())